# Setup

Pip install `ultralytics` and [dependencies](https://github.com/ultralytics/ultralytics/blob/main/pyproject.toml) and check software and hardware.

[![PyPI - Version](https://img.shields.io/pypi/v/ultralytics?logo=pypi&logoColor=white)](https://pypi.org/project/ultralytics/) [![Downloads](https://static.pepy.tech/badge/ultralytics)](https://pepy.tech/project/ultralytics) [![PyPI - Python Version](https://img.shields.io/pypi/pyversions/ultralytics?logo=python&logoColor=gold)](https://pypi.org/project/ultralytics/)

In [1]:
%pip install ultralytics
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.2.78  Python-3.8.10 torch-2.4.0+cu121 CUDA:0 (NVIDIA GeForce RTX 3060, 12287MiB)
Setup complete  (4 CPUs, 16.0 GB RAM, 16.8/251.8 GB disk)


# 1. Train

In [ ]:
# Train YOLOv8n 
# !yolo train model=yolov8n.pt data='./datasets/data.yaml' batch=32 epochs=300 imgsz=640 device=0

import torch
from ultralytics import YOLO
import os

def train(dataset_path=None, model = None, epochs=1, batch_size=16, imgsz=640):
    training_device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print("Using device:", training_device)

    batch_size = batch_size
    epochs = epochs
    imgsz = imgsz

    dataset_path = dataset_path
    model = YOLO(model)
    model.train(task='detect',
                data=dataset_path,
                epochs=epochs,
                batch=batch_size,
                imgsz=imgsz,
                device=training_device)

if __name__ == '__main__':
    currentDir = os.getcwd()

    #remove cache file before 
    datasetDir = os.path.join(currentDir, "datasets")
    os.remove(os.path.join(datasetDir, "train", "labels.cache"))
    os.remove(os.path.join(datasetDir, "val", "labels.cache"))

    
    datasetFile = os.path.join(datasetDir, "data.yaml")
    train(dataset_path=datasetFile, model = 'yolov8n.pt', epochs=300, batch_size=32, imgsz=640)

# 2. Val

In [2]:
# !yolo val model='./runs/detect/train/weights/best.pt' data='datasets/data.yaml'

import torch
from ultralytics import YOLO

def validate(dataset_path=None, model_path=None):
    validation_device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print("Using device:", validation_device)

    dataset_path = dataset_path
    model = YOLO(model_path)
    model.val(data=dataset_path, device=validation_device)

if __name__ == '__main__':
    currentDir = os.getcwd()
    datasetFile = os.path.join(currentDir, "datasets" , "data.yaml")
    trainedModel = os.path.join(currentDir, "runs", "detect", "train", "weights", "best.pt")
    validate(dataset_path=datasetFile, model_path=trainedModel)

Using device: cuda
Ultralytics YOLOv8.2.78  Python-3.8.10 torch-2.4.0+cu121 CUDA:0 (NVIDIA GeForce RTX 2060, 6144MiB)
Model summary (fused): 168 layers, 3,007,208 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning D:\HighwayCounter_YOLOv8\datasets\labels\valid.cache... 85 images, 0 backgrounds, 0 corrupt: 100%|██████████| 85/85 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 6/6 [00:01<00:00,  3.43it/s]


                   all         85        308      0.903      0.823      0.883      0.778
                   car         80        215      0.812      0.912      0.916      0.705
                   bus         15         15      0.998        0.8      0.858      0.811
                 truck         49         78      0.898      0.756      0.877      0.818
Speed: 1.6ms preprocess, 6.8ms inference, 0.0ms loss, 3.1ms postprocess per image
Results saved to runs\detect\val3


# 3. Predict


In [ ]:
!yolo predict model=trainedModel source='bike_car.mp4' device=0

# 4. Export

Export a YOLOv8 model to any supported format below with the `format` argument, i.e. `format=onnx`. See [YOLOv8 Export Docs](https://docs.ultralytics.com/modes/export/) for more information.

- 💡 ProTip: Export to [ONNX](https://docs.ultralytics.com/integrations/onnx/) or [OpenVINO](https://docs.ultralytics.com/integrations/openvino/) for up to 3x CPU speedup.  
- 💡 ProTip: Export to [TensorRT](https://docs.ultralytics.com/integrations/tensorrt/) for up to 5x GPU speedup.

| Format                                                                   | `format` Argument | Model                     | Metadata | Arguments                                                            |
|--------------------------------------------------------------------------|-------------------|---------------------------|----------|----------------------------------------------------------------------|
| [PyTorch](https://pytorch.org/)                                          | -                 | `yolov8n.pt`              | ✅        | -                                                                    |
| [TorchScript](https://docs.ultralytics.com/integrations/torchscript)     | `torchscript`     | `yolov8n.torchscript`     | ✅        | `imgsz`, `optimize`, `batch`                                         |
| [ONNX](https://docs.ultralytics.com/integrations/onnx)                   | `onnx`            | `yolov8n.onnx`            | ✅        | `imgsz`, `half`, `dynamic`, `simplify`, `opset`, `batch`             |
| [OpenVINO](https://docs.ultralytics.com/integrations/openvino)           | `openvino`        | `yolov8n_openvino_model/` | ✅        | `imgsz`, `half`, `int8`, `batch`                                     |
| [TensorRT](https://docs.ultralytics.com/integrations/tensorrt)           | `engine`          | `yolov8n.engine`          | ✅        | `imgsz`, `half`, `dynamic`, `simplify`, `workspace`, `int8`, `batch` |
| [CoreML](https://docs.ultralytics.com/integrations/coreml)               | `coreml`          | `yolov8n.mlpackage`       | ✅        | `imgsz`, `half`, `int8`, `nms`, `batch`                              |
| [TF SavedModel](https://docs.ultralytics.com/integrations/tf-savedmodel) | `saved_model`     | `yolov8n_saved_model/`    | ✅        | `imgsz`, `keras`, `int8`, `batch`                                    |
| [TF GraphDef](https://docs.ultralytics.com/integrations/tf-graphdef)     | `pb`              | `yolov8n.pb`              | ❌        | `imgsz`, `batch`                                                     |
| [TF Lite](https://docs.ultralytics.com/integrations/tflite)              | `tflite`          | `yolov8n.tflite`          | ✅        | `imgsz`, `half`, `int8`, `batch`                                     |
| [TF Edge TPU](https://docs.ultralytics.com/integrations/edge-tpu)        | `edgetpu`         | `yolov8n_edgetpu.tflite`  | ✅        | `imgsz`, `batch`                                                     |
| [TF.js](https://docs.ultralytics.com/integrations/tfjs)                  | `tfjs`            | `yolov8n_web_model/`      | ✅        | `imgsz`, `half`, `int8`, `batch`                                     |
| [PaddlePaddle](https://docs.ultralytics.com/integrations/paddlepaddle)   | `paddle`          | `yolov8n_paddle_model/`   | ✅        | `imgsz`, `batch`                                                     |
| [NCNN](https://docs.ultralytics.com/integrations/ncnn)                   | `ncnn`            | `yolov8n_ncnn_model/`     | ✅        | `imgsz`, `half`, `batch`                                             |

In [6]:
!yolo export model='./runs/detect/train/weights/best.pt' imgsz=640,640 format=onnx opset=12 simplify=True dynamic=False

Ultralytics YOLOv8.2.81 🚀 Python-3.8.6 torch-2.2.2+cu118 CPU (AMD Ryzen 5 2600 Six-Core Processor)
Model summary (fused): 168 layers, 3,007,208 parameters, 0 gradients, 8.1 GFLOPs

PyTorch: starting from 'runs\detect\train\weights\best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 12, 8400) (5.9 MB)

ONNX: starting export with onnx 1.16.1 opset 12...
ONNX: slimming with onnxslim 0.1.33...
ONNX: export success ✅ 2.8s, saved as 'runs\detect\train\weights\best.onnx' (11.7 MB)

Export complete (4.9s)
Results saved to D:\PROJECT\GitHub\THIGIACMAYTINH\HighwayCounter_YOLOv8\runs\detect\train\weights
Predict:         yolo predict task=detect model=runs\detect\train\weights\best.onnx imgsz=640  
Validate:        yolo val task=detect model=runs\detect\train\weights\best.onnx imgsz=640 data=./datasets/data.yaml  
Visualize:       https://netron.app
💡 Learn more at https://docs.ultralytics.com/modes/export
